# Example 6: Aperture photometry with Photutils

Máster en Astrofísica UCM
Técnicas Experimentales en Astrofísica

Jaime Zamorano, Nicolás Cardiel and Sergio Pascual

This notebook has reproduced parts of 

Photoutils Aperture Photometry 
    https://photutils.readthedocs.io/en/stable/aperture.html

Photoutils Source Detection
    https://photutils.readthedocs.io/en/stable/detection.html
 
v2 2020/05/20
v3 2021/01/22

In [ ]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.patches import Rectangle

from astropy.io import fits
from astropy.stats import sigma_clip, sigma_clipped_stats, mad_std

import ccdproc
from ccdproc import CCDData, Combiner
from ccdproc import median_filter

import numpy as np
#import numpy.ma as ma

from photutils import DAOStarFinder
from photutils import find_peaks
from photutils import CircularAperture
from photutils import CircularAnnulus
from photutils import aperture_photometry

In [ ]:
# Some style for better looking plots
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Verdana']
plt.rcParams['font.size'] = 12
plt.rcParams['font.size'] = 12
plt.rcParams['lines.linewidth'] = 4.
plt.rcParams['axes.labelsize'] = 'medium'
plt.rcParams['grid.linewidth'] = 1.0
plt.rcParams['grid.linestyle'] = '-'
plt.rcParams['xtick.minor.size']=4
plt.rcParams['xtick.major.size']=8
plt.rcParams['ytick.minor.size']=4
plt.rcParams['ytick.major.size']=8
plt.rcParams['figure.figsize'] = 12,6
plt.rcParams['figure.subplot.bottom'] = 0.15
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['xtick.labelsize'] = 10

Reading and displaying the image data and a region to be used in the first part of the example

In [ ]:
image = fits.getdata('./FITS_files/ucmP_0050.fits')
#x1, x2, y1, y2 = 100, 500, 300, 700
x1, x2, y1, y2 = 800, 1000, 800, 1000
ximage = image[x1:x2, y1:y2]
vmin,vmax = 1000,10000
fig, axarr = plt.subplots(ncols=2, nrows=1, figsize=(14, 9))
axarr[0].imshow(image, cmap='gray', vmin=vmin, vmax=vmax,norm=LogNorm())
axarr[1].imshow(ximage, cmap='gray', vmin=vmin, vmax=vmax,norm=LogNorm())

## Aperture photometry of some stars in the image

Photutils also includes a find_peaks() function to find local peaks in an image that are above a specified threshold value. Peaks are the local maxima above a specified threshold that are separated by a specified minimum number of pixels.

In [ ]:
mean, median, std = sigma_clipped_stats(ximage, sigma=3.0)
threshold = median + (10. * std)
tbl = find_peaks(ximage, threshold, box_size=40)
tbl['peak_value'].info.format = '%.8g'  # for consistent table output
print(tbl[:10])  # print only the first 10 peaks

Displaying the peaks found as circular apertures

In [ ]:
radius = 10
vmin, vmax = 3000, 6000
positions = np.transpose((tbl['x_peak'], tbl['y_peak']))
apertures = CircularAperture(positions, r=radius)
plt.imshow(ximage, cmap='gray', vmin=vmin, vmax=vmax, norm=LogNorm())
apertures.plot(color='#0547f9', lw=1.5)

We are ready to perform a simple aperture photometry summing up all the counts inside the circular apertures centered in the sources found

In [ ]:
phot_table = aperture_photometry(ximage, apertures)
phot_table['aperture_sum'].info.format = '%.4g'  # for consistent table output
print(phot_table)

The sources can be added if their positions are known or estimated.

In [ ]:
positions_new = [(113, 150), (73, 140), (100, 125)]
apertures_new = CircularAperture(positions_new, r=radius)
plt.imshow(ximage, cmap='gray', vmin=vmin, vmax=vmax, norm=LogNorm())
apertures.plot(color='#0547f9', lw=1.5)
apertures_new.plot(color='red', lw=1.5)

We have selected a position without any source to measure the sky background in this area.

In [ ]:
phot_table = aperture_photometry(ximage, apertures_new)
phot_table['aperture_sum'].info.format = '%.4g'  # for consistent table output
print(phot_table)

Notice that the background should be subtracted from the raw counts to determine the net counts of the sources.

## Selecting several sources 

Let select more stars of the original image using DAOStarFinder. In order to avoid the central part of the image with both galaxies we will mask this region

In [ ]:
# Region to be masked during DAOStarFinder searches
mask = np.zeros(image.shape, dtype=bool)
x1, x2, y1, y2= 400, 1000, 200, 1200
mask[x1:x2, y1:y2] = True

In [ ]:
plt.figure(figsize=(8, 8))
plt.imshow(image, cmap='gray', vmin=vmin, vmax=vmax) #, norm=LogNorm())
currentAxis = plt.gca()
currentAxis.add_patch(Rectangle((y1,x1), (y2-y1), (x2-x1),
                      alpha=0.4, facecolor='yellow'))
plt.text(300,600,'area to be masked', fontsize=15)
plt.text(300,700,'(no star searching here)', fontsize=15)

In [ ]:
# only sources outside the mask region are found
daofind = DAOStarFinder(fwhm=5.0, threshold=10.*std)  
sources = daofind(image - median, mask=mask)  
for col in sources.colnames:  
     sources[col].info.format = '%.8g'  # for consistent table output
print(sources) 

Let display the stars found 

In [ ]:
positions = np.transpose((sources['xcentroid'], sources['ycentroid']))
positions = positions.round(decimals=2)
#print(positions)
apertures = CircularAperture(positions, r=10.)
plt.figure(figsize=(8,8))
plt.imshow(image, cmap='gray', vmin=vmin, vmax=vmax) #, norm=LogNorm())
apertures.plot(color='red', lw=1.5, alpha=0.5)
currentAxis = plt.gca()
currentAxis.add_patch(Rectangle((y1,x1), (y2-y1), (x2-x1),
                      alpha=0.1, facecolor='yellow'))

### Simple aperture photometry on the detected sources

In [ ]:
radius = 10
aperture = CircularAperture(positions, r=radius)
phot_table = aperture_photometry(image, aperture)
phot_table['aperture_sum'].info.format = '%.4g'  # for consistent table output
print(phot_table)

The last row contents the total counts inside the aperture centered in the detected stars. These values are not corrected from the background. 

### Global Background Subtraction
We can made a rougth estimate the background level using the median value for the whole image or better using a region without sources.

In [ ]:
median     = np.median(image)
median_sky = np.median(image[0:200,800:1000])
print('median', median, ' sky ', median_sky)

In [ ]:
bkg = median_sky
phot_table = aperture_photometry(image - bkg, aperture) 
phot_table['aperture_sum'].info.format = '%.4g'  # for consistent table output
print(phot_table)

It is posible to define several circular apertures and perform  

In [ ]:
radii = [10., 15., 20.]
apertures = [CircularAperture(positions, r=r) for r in radii]
phot_table = aperture_photometry(image, apertures)
for col in phot_table.colnames:
     phot_table[col].info.format = '%.8g'  # for consistent table output
print(phot_table)

### Local Background Subtraction
Classical aperture photometry uses a circular annulus around the star image to determine the background. We will select radius of 8 pixels for the aperture and a circular annulus of inner radius of 12 and outer radius of 20. 

In [ ]:
aperture = CircularAperture(positions, r=8)
annulus_aperture = CircularAnnulus(positions, r_in=12., r_out=20.)
plt.figure(figsize=(8,8))
plt.imshow(image[0:300, 0:500], cmap='gray', vmin=vmin, vmax=vmax)
aperture.plot(color='white', lw=1)
annulus_aperture.plot(color='red', lw=1)

In [ ]:
apers = [aperture, annulus_aperture]
phot_table = aperture_photometry(image, apers)
for col in phot_table.colnames:
    phot_table[col].info.format = '%.8g'  # for consistent table output
print(phot_table)

The last two columns show the total number of counts inside the cicle and the circular annulus. 
The areas of the apertures are different and we should correct before substracting the background by scaling the results for the same area

In [ ]:
print(aperture.area, annulus_aperture.area)

In [ ]:
bkg_mean = phot_table['aperture_sum_1'] / annulus_aperture.area
bkg_sum  = bkg_mean * aperture.area
final_sum = phot_table['aperture_sum_0'] - bkg_sum
phot_table['residual_aperture_sum'] = final_sum
phot_table['residual_aperture_sum'].info.format = '%.8g'  # for consistent table output
#print(phot_table['residual_aperture_sum'])  

In [ ]:
for col in phot_table.colnames:
    phot_table[col].info.format = '%.8g'  # for consistent table output
print(phot_table)

The last column shows the net counts obtained after substracting the scaled background